In [1]:
import pandas as pd
import glob
import os
from importlib.machinery import SourceFileLoader

## Load data and modules

Import egemaps extraction module

In [44]:
code_path='/home/gbarchi/Documentos/Trust/OCEAN-TRUST/src/extract_egemaps.py'
extract_egemaps = SourceFileLoader("extract_features", code_path).load_module() 

Import utils

In [53]:
format_code_path='/home/gbarchi/Documentos/Trust/OCEAN-TRUST/src/utils.py'
utils=SourceFileLoader("format_data", format_code_path).load_module() 

Import modeling

In [67]:
modeling_code_path='/home/gbarchi/Documentos/Trust/OCEAN-TRUST/src/modeling.py'
modeling=SourceFileLoader("cross_val", modeling_code_path).load_module() 

Load manual annotations

In [31]:
music_annot_df=pd.read_csv('../data/manual_music_annotations/df_music_labels.csv',index_col=0)
no_music_list=music_annot_df[music_annot_df['music']==0].name
music_list=music_annot_df[music_annot_df['music']==1].name

In [32]:
no_speech_files='/home/gbarchi/Documentos/Trust/OCEAN-TRUST/data/silero/silero_no_speech'

no_music_paths=[]
for dir_nm in no_music_list:
    no_music_paths.append(glob.glob(os.path.join(no_speech_files,'*',dir_nm))[0])

music_paths=[]
for dir_m in music_list:
    music_paths.append(glob.glob(os.path.join(no_speech_files,'*',dir_m))[0])

## Extract egemaps

In [36]:
no_music_features_path='/home/gbarchi/Documentos/Trust/OCEAN-TRUST/data/features/egemaps_silero_no_speech_no_music_manual_annot.csv'
music_features_path='/home/gbarchi/Documentos/Trust/OCEAN-TRUST/data/features/egemaps_silero_no_speech_music_manual_annot.csv'

Extract no music features

In [41]:
audio_duration=2

In [45]:
extract_egemaps.extract_features(no_music_features_path,no_music_paths,audio_duration)

32000


100%|██████████| 146/146 [00:12<00:00, 11.50it/s]


Extract music features

In [46]:
extract_egemaps.extract_features(music_features_path,music_paths,audio_duration)

32000


100%|██████████| 170/170 [00:14<00:00, 11.37it/s]


## Prepare data

In [56]:
feat_df_nm=pd.read_csv('../data/features/egemaps_silero_no_speech_no_music_manual_annot.csv')
feat_df_m=pd.read_csv('../data/features/egemaps_silero_no_speech_music_manual_annot.csv')
labels_df=pd.read_csv('../data/labels/new_partitions-labels.csv')

No speech + No music

In [55]:
df_no_music=utils.format_data(feat_df_nm,labels_df)

No speech + Music

In [57]:
df_music=utils.format_data(feat_df_m,labels_df)

## Train models

In [61]:
label_tags=['extraversion', 'neuroticism','agreeableness', 'conscientiousness', 'openness']
feature_tags=feat_df_m.columns[~feat_df_m.columns.isin(['Name','Part'])]

In [68]:
df_metrics_music,feature_importance_music,y_val_music=modeling.cross_val(df_music,feature_tags,label_tags,n_folds=5)

/home/gbarchi/Documentos/Trust/OCEAN-TRUST/src/modeling.py:387: RuntimeWarning: invalid value encountered in sqrt
  metrics=[r2_all,np.sqrt(r2_all),MAE_all,MSE_all,RMSE_all,fold]
/home/gbarchi/Documentos/Trust/OCEAN-TRUST/src/modeling.py:387: RuntimeWarning: invalid value encountered in sqrt
  metrics=[r2_all,np.sqrt(r2_all),MAE_all,MSE_all,RMSE_all,fold]
/home/gbarchi/Documentos/Trust/OCEAN-TRUST/src/modeling.py:387: RuntimeWarning: invalid value encountered in sqrt
  metrics=[r2_all,np.sqrt(r2_all),MAE_all,MSE_all,RMSE_all,fold]
/home/gbarchi/Documentos/Trust/OCEAN-TRUST/src/modeling.py:387: RuntimeWarning: invalid value encountered in sqrt
  metrics=[r2_all,np.sqrt(r2_all),MAE_all,MSE_all,RMSE_all,fold]
/home/gbarchi/Documentos/Trust/OCEAN-TRUST/src/modeling.py:387: RuntimeWarning: invalid value encountered in sqrt
  metrics=[r2_all,np.sqrt(r2_all),MAE_all,MSE_all,RMSE_all,fold]


In [69]:
df_metrics_music

,r2,r,MAE,MSE,RMSE,fold
0,-0.135916,NaN,0.101836,0.016525,0.128550,0.0
1,-0.110177,NaN,0.105347,0.016802,0.129624,1.0
2,-0.030345,NaN,0.097618,0.014405,0.120021,2.0
3,-0.196946,NaN,0.099877,0.015481,0.124422,3.0
4,-0.029778,NaN,0.100988,0.015955,0.126312,4.0


In [70]:
df_metrics_no_music,feature_importance_no_music,y_val_no_music=modeling.cross_val(df_no_music,feature_tags,label_tags,n_folds=5)

/home/gbarchi/Documentos/Trust/OCEAN-TRUST/src/modeling.py:387: RuntimeWarning: invalid value encountered in sqrt
  metrics=[r2_all,np.sqrt(r2_all),MAE_all,MSE_all,RMSE_all,fold]
/home/gbarchi/Documentos/Trust/OCEAN-TRUST/src/modeling.py:387: RuntimeWarning: invalid value encountered in sqrt
  metrics=[r2_all,np.sqrt(r2_all),MAE_all,MSE_all,RMSE_all,fold]
/home/gbarchi/Documentos/Trust/OCEAN-TRUST/src/modeling.py:387: RuntimeWarning: invalid value encountered in sqrt
  metrics=[r2_all,np.sqrt(r2_all),MAE_all,MSE_all,RMSE_all,fold]
/home/gbarchi/Documentos/Trust/OCEAN-TRUST/src/modeling.py:387: RuntimeWarning: invalid value encountered in sqrt
  metrics=[r2_all,np.sqrt(r2_all),MAE_all,MSE_all,RMSE_all,fold]


In [71]:
df_metrics_no_music

,r2,r,MAE,MSE,RMSE,fold
0,0.033979,0.184333,0.098780,0.015315,0.123754,0.0
1,-0.230394,NaN,0.121857,0.022787,0.150955,1.0
2,-0.418557,NaN,0.123974,0.022795,0.150980,2.0
3,-0.150533,NaN,0.122507,0.024953,0.157964,3.0
4,-0.262666,NaN,0.130361,0.024394,0.156185,4.0
